In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data/gemstone.csv')

In [3]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df=df.drop(['id'],axis=1)

In [5]:
X=df.drop(['price'],axis=1)
y=df[['price']]

In [6]:
cat_col=X.select_dtypes(include='object').columns
num_col=X.select_dtypes(exclude='object').columns

In [7]:
cut_cat=['Fair','Good','Very Good','Premium','Ideal']
color_cat=['D','E','F','G','H','I','J']
clarity_cat=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [8]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [9]:
num_pipe=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

In [10]:
cat_pipe=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('encoder',OrdinalEncoder(categories=[cut_cat,color_cat,clarity_cat])),
        ('scaler',StandardScaler())
    ]
)

In [11]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipe,num_col),
    ('cat_piepline',cat_pipe,cat_col)
])

In [12]:
from sklearn.model_selection import train_test_split as tts

In [13]:
Xtr,Xte,ytr,yte=tts(X,y,random_state=30,test_size=0.3)

In [14]:
Xtr=pd.DataFrame(preprocessor.fit_transform(Xtr),columns=preprocessor.get_feature_names_out())
Xte=pd.DataFrame(preprocessor.transform(Xte),columns=preprocessor.get_feature_names_out())

In [15]:
Xtr.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_piepline__cut,cat_piepline__color,cat_piepline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731


In [46]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [39]:
from sklearn.metrics import r2_score as r2, mean_absolute_error as mae, mean_squared_error as mse

In [40]:
reg=LinearRegression()

In [41]:
reg.fit(Xtr,ytr)

LinearRegression()

In [42]:
reg.coef_

array([[ 6433.66003594,  -132.75843566,   -70.42922179, -1720.30971463,
         -499.29302619,   -63.39317848,    72.44537247,  -460.41604642,
          650.76431652]])

In [43]:
reg.intercept_

array([3970.76628955])

In [44]:
import numpy as np
def eval_model(yte,y_pred):
    rmse=np.sqrt(mse(y_pred,yte))
    r2_sq=r2(y_pred,yte)
    return r2_sq,mae(y_pred,yte),rmse

In [47]:
models={
    'linear_reg':LinearRegression(),
    'lasso':Lasso(),
    'ridge':Ridge(),
    'elas':ElasticNet(),
    'dtree':DecisionTreeRegressor(),
    'rforest':RandomForestRegressor()
}

trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(models)):
    model=list(models.values())[i]
    model.fit(Xtr,ytr)
    y_pred=model.predict(Xte)
    r2_sq,mean,root=eval_model(yte,y_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print('Model Performance ')
    print('RMSE: ',root)
    print('MAE: ',mean)
    print('R2 Score: ',r2_sq*100)
    r2_list.append(r2)
    print('='*35)
    print('\n')

linear_reg
Model Performance 
RMSE:  1013.9047094344003
MAE:  674.025511579685
R2 Score:  93.25284792033962


lasso
Model Performance 
RMSE:  1013.8784226767013
MAE:  675.0716923362161
R2 Score:  93.2450057309906


ridge
Model Performance 
RMSE:  1013.9059272771599
MAE:  674.055580079832
R2 Score:  93.25254009213401


elas
Model Performance 
RMSE:  1533.4162456064046
MAE:  1060.7368759154729
R2 Score:  78.04307330764672


dtree
Model Performance 
RMSE:  839.3933967901208
MAE:  423.67153418744545
R2 Score:  95.66343615172659




C:\Users\ASUS\AppData\Local\Temp\ipykernel_9848\3508256408.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(Xtr,ytr)


rforest
Model Performance 
RMSE:  610.9319422958653
MAE:  311.6018366809838
R2 Score:  97.65829624382907


